In [1]:
# import SAGA code

import SAGA
from SAGA import ObjectCuts as C
from SAGA.utils import get_sdss_bands, get_sdss_colors
from SAGA.targets import TargetSelection, calc_simple_satellite_probability, calc_gmm_satellite_probability

print('SAGA code version', SAGA.__version__)

SAGA code version 0.5.4


In [2]:
# import other modules

from collections import defaultdict
import numpy as np
from scipy.stats import poisson, norm
from astropy.table import Table
from easyquery import Query

In [3]:
# initialize SAGA objects (Database, HostCatalog, ObjectCatalog)
# NOTE: change the path of `root_dir` to your SAGA dropbox path

saga_database = SAGA.Database(root_dir='/home/yymao/Dropbox/Academia/Collaborations/SAGA')
saga_database.base_file_path_pattern = '/home/yymao/Documents/Research/SAGA/new_base_catalogs/base_dr14_nsa{}.fits.gz'
saga_host_catalog = SAGA.HostCatalog(saga_database)
saga_object_catalog = SAGA.ObjectCatalog(saga_database)

## check completeness

In [13]:
# specify columns to load
columns = 'OBJID HOST_NSAID HOST_RA RA DEC SATS SPEC_Z ZQUALITY'.split()
columns.extend(map('{}_mag'.format, get_sdss_bands()))
columns.extend(map('{}_err'.format, get_sdss_bands()))
columns.extend(get_sdss_colors())
columns.extend(map('{}_err'.format, get_sdss_colors()))

In [14]:
# get host ids (in a particular order)

hosts = saga_host_catalog.load()
hosts_other = hosts[np.isin(hosts['NSAID'], saga_host_catalog.resolve_id('paper1'), True, True)]
del hosts_other['coord']
hosts_other.sort('RA')
host_ids = saga_host_catalog.resolve_id('paper1_complete') + saga_host_catalog.resolve_id('paper1_incomplete') + list(hosts_other['NSAID'])

In [6]:
def normalize_pgmm(x):
    return np.exp((x-1)*10.0)*0.1

In [23]:
gmm_parameters = saga_database['gmm_parameters'].read()
data = defaultdict(list)

for t in saga_object_catalog.load(host_ids, cuts=(C.basic_cut & C.gri_cut & (~C.has_spec)), columns=columns, return_as='iter'):
    if not len(t):
        continue

    host_id = t['HOST_NSAID'][0]
    try:
        saga_name = saga_host_catalog.id_to_name(host_id)
    except KeyError:
        saga_name = ''
    data['SAGA_name'].append(saga_name)
    data['NSAID'].append(host_id)
    data['RA'].append(t['HOST_RA'][0])

    data['no_spec'].append(len(t))
    no_spec_bright = C.sdss_limit.count(t)
    data['no_spec_bright'].append(no_spec_bright)
    
    t = (~C.sdss_limit).filter(t)
    ppca = calc_simple_satellite_probability(t).sum()
    pgmm_orig = calc_gmm_satellite_probability(t, gmm_parameters)
    pgmm = normalize_pgmm(pgmm_orig)
    
    data['high_pgmm'].append(np.count_nonzero(pgmm_orig > 0.9))
    data['miss_pca'].append(ppca.sum())
    data['miss_gmm'].append(pgmm.sum())
    
    pgmm.sort()
    for n in (100, 200):
        nf = n - no_spec_bright
        if nf < 0:
            m = pgmm.sum()
        elif nf > len(t):
            m = 0
        else:
            m = pgmm[:-nf].sum()
        data['miss_gmm_{}'.format(n)].append(m)

data = Table(data)

/home/yymao/miniconda3/lib/python3.6/site-packages/SAGA/targets/assign_targeting_score.py:13: RuntimeWarning: overflow encountered in exp
  return np.where(x > model_parameters[2], np.minimum(np.exp((x-model_parameters[3])*model_parameters[4]), model_parameters[5]), 0.0)


In [24]:
for name in data.colnames:
    if name.startswith('miss_'):
        data[name].format = '%.3f'

data.pprint(-1, -1)

 SAGA_name  NSAID     RA    no_spec no_spec_bright high_pgmm miss_pca miss_gmm miss_gmm_100 miss_gmm_200
----------- ------ -------- ------- -------------- --------- -------- -------- ------------ ------------
  Gilgamesh 166313  234.132      27              0         0    0.300    0.003        0.000        0.000
    Odyssey 147100  248.087      20              0         1    0.060    0.055        0.000        0.000
       Dune 165536  221.546      40              0         3    0.290    0.300        0.000        0.000
       AnaK  61945  354.131      56              0         2    0.439    0.178        0.000        0.000
     Narnia 132339  39.5482      74              1         0    0.062    0.003        0.000        0.000
   OBrother 149781  335.913      83              0         3    0.102    0.196        0.000        0.000
   StarTrek  33446  123.241     119              0         4    0.471    0.353        0.000        0.000
    Catch22 150887  348.683     158              2     

In [25]:
data['no_spec no_spec_bright high_pgmm miss_pca miss_gmm miss_gmm_100 miss_gmm_200'.split()][16:].to_pandas().describe()

,no_spec,no_spec_bright,high_pgmm,miss_pca,miss_gmm,miss_gmm_100,miss_gmm_200
count,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000
mean,1232.509091,14.909091,24.036364,2.103851,2.336221,0.307952,0.044997
std,523.941892,17.802697,14.507934,1.067189,1.287819,0.665683,0.108829
min,578.000000,1.000000,4.000000,0.799616,0.453792,0.003399,0.001798
25%,831.000000,4.500000,14.000000,1.372078,1.430530,0.012499,0.003077
50%,1083.000000,7.000000,20.000000,1.838845,2.045949,0.042667,0.005488
75%,1457.500000,16.500000,30.000000,2.523072,2.880571,0.212352,0.021219
max,2652.000000,79.000000,78.000000,5.777630,7.048063,3.037454,0.530518
